In [12]:
import datasets
import matplotlib.pyplot as plt
import numpy as np
import pytorch_lightning as pl
import torch
from torch.utils.data import TensorDataset, DataLoader
from prosenet import Encoder, EncoderClassifier, ProSeNet, PrototypeProjection
from sklearn.utils import class_weight

In [13]:
# Dataset parameters
num_classes = 5
num_features = 1
gpus = 1
device = torch.device("cuda")

In [14]:
# Set any non-default args
new_rnn_args = {
    "input_size": num_features,
    "hidden_size": 32,
    "num_layers": 3,
    "dropout": 0.1,
    "bidirectional": True,
    "batch_first": True,
}

encoder = Encoder(new_rnn_args)

new_proto_args = {
    "K": 30,
    "D": 32,
    "dmin": 2.0,
    "Ld": 0.01,  # 0.1,
    "Lc": 0.0,
    "Le": 1.0,
}

In [15]:
data = datasets.ArrhythmiaDataset("../dataset/data/", normalize=False)
print(data)

MIT-BIH Arrhythmia Dataset
Num classes: 5
Input shape: (187, 1)
Train, Test counts: 87554, 21892



In [16]:
train_gen = DataLoader(
    TensorDataset(torch.FloatTensor(data.X_train), torch.LongTensor(data.y_train)),
    batch_size=128,
    shuffle=True,
    num_workers=4,
)
test_gen = DataLoader(
    TensorDataset(torch.FloatTensor(data.X_test), torch.LongTensor(data.y_test)),
    batch_size=128,
    shuffle=False,
    num_workers=4,
)


class_weights = 1 - (np.bincount(data.y_train) / data.y_train.shape[0])
class_weights = torch.FloatTensor(class_weights).to(device)
# class_weights[0] = 0.01
class_weights

tensor([0.1723, 0.9746, 0.9339, 0.9927, 0.9265], device='cuda:0')

# First train just the `encoder`

In [17]:
encoder_classifier = EncoderClassifier(
    encoder, class_weights, new_rnn_args["hidden_size"], num_classes
)
print(encoder_classifier)

EncoderClassifier(
  (encoder): Encoder(
    (rnn): LSTM(1, 32, num_layers=3, batch_first=True, dropout=0.1, bidirectional=True)
  )
  (pred): Linear(in_features=32, out_features=5, bias=True)
  (named_metrics_train): ModuleDict(
    (acc): Accuracy()
    (avg_p): Precision()
    (avg_r): Recall()
  )
  (named_metrics_val): ModuleDict(
    (acc): Accuracy()
    (avg_p): Precision()
    (avg_r): Recall()
  )
)


In [18]:
tb_logger = pl.loggers.TensorBoardLogger(
    name="ecg_encoder", save_dir="lightning_logs/"
)
trainer = pl.Trainer(
    max_epochs=100,
    gpus=gpus,
    deterministic=True,
    logger=tb_logger,
)
trainer.fit(encoder_classifier, train_gen, test_gen)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name                | Type       | Params
---------------------------------------------------
0 | encoder             | Encoder    | 59.1 K
1 | pred                | Linear     | 165   
2 | named_metrics_train | ModuleDict | 0     
3 | named_metrics_val   | ModuleDict | 0     
---------------------------------------------------
59.3 K    Trainable params
0         Non-trainable params
59.3 K    Total params
0.237     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

# Now freeze the `encoder` and train the prototypes head

In [19]:
for para in encoder.parameters():
    para.requires_grad = False
# print(para)

In [20]:
pnet = ProSeNet(
    n_classes=num_classes,
    class_weights=class_weights,
    encoder=encoder,
    prototypes_args=new_proto_args,
)

pnet

ProSeNet(
  (encoder): Encoder(
    (rnn): LSTM(1, 32, num_layers=3, batch_first=True, dropout=0.1, bidirectional=True)
  )
  (prototypes_layer): Prototypes()
  (classifier): Sequential(
    (0): Linear(in_features=30, out_features=5, bias=True)
  )
  (named_metrics_train): ModuleDict(
    (acc): Accuracy()
    (avg_p): Precision()
    (avg_r): Recall()
  )
  (named_metrics_val): ModuleDict(
    (acc): Accuracy()
    (avg_p): Precision()
    (avg_r): Recall()
  )
)

In [21]:
trainer = pl.Trainer(
    max_epochs=100,
    gpus=gpus,
    deterministic=True,
    gradient_clip_val=5.0,
    callbacks=[PrototypeProjection(pnet, train_gen)],
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [22]:
trainer.fit(pnet, train_gen, test_gen)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name                | Type       | Params
---------------------------------------------------
0 | encoder             | Encoder    | 59.1 K
1 | prototypes_layer    | Prototypes | 960   
2 | classifier          | Sequential | 155   
3 | named_metrics_train | ModuleDict | 0     
4 | named_metrics_val   | ModuleDict | 0     
---------------------------------------------------
1.1 K     Trainable params
59.1 K    Non-trainable params
60.3 K    Total params
0.241     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

new protos tensor([[86532,  8370, 68886, 37615, 45996, 41031, 19912, 68124, 46040, 36394,
         54249, 67008, 34436, 12771, 26819, 11464, 61069, 57662, 54883, 14655,
         40396, 21156, 54925, 20992, 65537, 46327, 67828, 38997, 68195, 70891]],
       device='cuda:0')
... assigned new prototypes from projections.


# Diagnostics

In [ ]:
# Encoding layer(s) output
pnet.encoder(torch.FloatTensor(data.X_train[np.newaxis, 0]))

In [ ]:
# Prototype layer weights
protos = np.squeeze(pnet.prototypes_layer.prototypes.detach().cpu().numpy())
print(protos.min(), protos.max())
print(protos.shape)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
cax = ax.matshow(protos, cmap=plt.cm.RdBu)
fig.colorbar(cax, ax=ax)

In [ ]:
pred_weights = np.squeeze(pnet.classifier[0].weight.detach().cpu().numpy())
print(pred_weights.min(), pred_weights.max())

fig, ax = plt.subplots(figsize=(10, 10))
cax = ax.matshow(pred_weights, cmap=plt.cm.RdBu)
fig.colorbar(cax, ax=ax)

In [ ]:
X_encoded = []
for batch in train_gen:
    X, y = batch
    X_encoded.append(encoder.to(device)(X.to(device)))
X_encoded = torch.cat(X_encoded, dim=0).unsqueeze(-2)
X_encoded = X_encoded.unsqueeze(-2).detach().cpu()
X_encoded

In [ ]:
# distance matrix from protos
# protos = pnet.prototypes_layer.weights[0]
d2 = torch.norm(X_encoded - protos, p=2, dim=-1)

idxs = d2.argmin(0).numpy()
idxs

In [ ]:
matched_protos = data.X_train[np.squeeze(idxs)]
matched_protos_y = data.y_train[np.squeeze(idxs)]
matched_protos.shape

In [ ]:
for i in range(30):
    plt.plot(np.arange(187), matched_protos[i, :, 0])
    plt.title(matched_protos_y[i])
    plt.show()

In [ ]:
pnet.predict(data.X_test[-3:, :], batch_size=1)

In [ ]:
data.y_test[-3:]

In [ ]:
plt.plot(np.arange(187), data.X_train[-2])

In [ ]:
class_weights

In [ ]:
pnet.classifier.weights[0].numpy()

In [ ]:
encoder_classifier.layers[-1].weights

In [ ]:
protos = np.squeeze(pnet.prototypes_layer.weights[0].numpy())
protos

In [ ]:
plt.matshow(protos, cmap=plt.cm.RdBu)

In [ ]:
pnet.encoder.predict(data.X_train[-2:, :])

In [ ]:
pnet.encoder.predict(data.X_train[:2, :])

In [ ]:
pnet.prototypes_layer._diversity_term()

In [ ]:
pnet.losses